In [ ]:
import pandas as pd
import numpy as np
from requests import get
from pandas import json_normalize
import time
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
form sklearn.pipeline import Pipeline
from sklear.feature_selection import SelectFromModel

In [15]:
modeling_data = pd.read_csv('/Users/patricknaylor/Desktop/CA_traffic/Data/selected_columns.csv')

In [16]:
print(modeling_data.nunique())

latitude                           281296
longitude                          315729
statewide_vehicle_type_at_fault        15
alcohol_involved                        2
truck_collision                         2
motorcycle_collision                    2
bicycle_collision                       2
pedestrian_collision                    2
lighting                                5
road_surface                            5
hit_and_run                             3
party_count                            14
injured_victims                        27
killed_victims                          7
location_type                           3
county_location                        58
population                              8
county_location_population             58
dtype: int64


In [17]:
def absolute_maximum_norm(column):
    return column / column.abs().max()

norm_columns = ['latitude', 'longitude', 'county_location_population']

for col in norm_columns:
    modeling_data[col] = absolute_maximum_norm(modeling_data[col])


In [18]:
modeling_data = pd.get_dummies(modeling_data, columns = ['alcohol_involved', 'statewide_vehicle_type_at_fault', 'truck_collision', 'motorcycle_collision', 'bicycle_collision', 'lighting',  'population', 'county_location', 'location_type', 'hit_and_run', 'road_surface'])


In [19]:
modeling_data['mortality_rate'] = modeling_data['killed_victims']/modeling_data['party_count']
modeling_data['injury_rate'] = modeling_data['injured_victims']/modeling_data['party_count']
modeling_data['is_injured'] = modeling_data['injured_victims'] != 0
modeling_data['is_injured'] = modeling_data['is_injured'].astype(int)

In [20]:
columns_list = list(modeling_data.columns)
removals = ['injured_victims', 'killed_victims',  'mortality_rate', 'injury_rate', 'is_injured', 'latitude', 'longitude']
for rem in removals:
    columns_list.remove(rem)
model_x = np.array(modeling_data[columns_list])
model_y = np.array(modeling_data['is_injured'])

In [21]:
x_oversample, y_oversample = resample(model_x[model_y == 0], model_y[model_y == 0], replace=True, n_samples=model_x[model_y ==1].shape[0])
X = np.vstack((model_x[model_y == 1], x_oversample))
y = np.hstack((model_y[model_y == 1], y_oversample))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
rfc = RandomForestClassifier()
parameters = {'n_estimators': [50, 100, 500], 'max_depth': [2, 5, 10, 30], 'min_samples_split': [2, 5, 10]}
clf = GridSearchCV(rfc, parameters, verbose=3)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END max_depth=2, min_samples_split=2, n_estimators=50;, score=0.610 total time=   2.2s
[CV 2/5] END max_depth=2, min_samples_split=2, n_estimators=50;, score=0.607 total time=   2.2s
[CV 3/5] END max_depth=2, min_samples_split=2, n_estimators=50;, score=0.606 total time=   2.3s
[CV 4/5] END max_depth=2, min_samples_split=2, n_estimators=50;, score=0.607 total time=   2.3s
[CV 5/5] END max_depth=2, min_samples_split=2, n_estimators=50;, score=0.612 total time=   2.2s
[CV 1/5] END max_depth=2, min_samples_split=2, n_estimators=100;, score=0.601 total time=   4.4s
[CV 2/5] END max_depth=2, min_samples_split=2, n_estimators=100;, score=0.602 total time=   4.4s
[CV 3/5] END max_depth=2, min_samples_split=2, n_estimators=100;, score=0.608 total time=   4.4s
[CV 4/5] END max_depth=2, min_samples_split=2, n_estimators=100;, score=0.613 total time=   4.4s
[CV 5/5] END max_depth=2, min_samples_split=2, n_estimators=100;, scor

[CV 5/5] END max_depth=5, min_samples_split=10, n_estimators=100;, score=0.623 total time=   7.5s
[CV 1/5] END max_depth=5, min_samples_split=10, n_estimators=500;, score=0.623 total time=  37.0s
[CV 2/5] END max_depth=5, min_samples_split=10, n_estimators=500;, score=0.620 total time=  37.2s
[CV 3/5] END max_depth=5, min_samples_split=10, n_estimators=500;, score=0.615 total time=  37.1s
[CV 4/5] END max_depth=5, min_samples_split=10, n_estimators=500;, score=0.621 total time=  36.9s
[CV 5/5] END max_depth=5, min_samples_split=10, n_estimators=500;, score=0.618 total time=  37.0s
[CV 1/5] END max_depth=10, min_samples_split=2, n_estimators=50;, score=0.629 total time=   5.7s
[CV 2/5] END max_depth=10, min_samples_split=2, n_estimators=50;, score=0.631 total time=   5.6s
[CV 3/5] END max_depth=10, min_samples_split=2, n_estimators=50;, score=0.627 total time=   5.8s
[CV 4/5] END max_depth=10, min_samples_split=2, n_estimators=50;, score=0.630 total time=   5.7s
[CV 5/5] END max_depth=1

In [ ]:
print(clf.best_params_)

In [ ]:

pip = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
  ('classification', RandomForestClassifier())
])
pip.fit(X_train, y_train)

In [27]:
reg = RandomForestClassifier(n_estimators=100, max_depth = 4)
reg.fit(X_train, y_train)
pred_train = reg.predict(X_train)
pred_test = reg.predict(X_test)


In [28]:
print(classification_report(y_train, pred_train))
print(classification_report(y_test, pred_test))
print(1-np.mean(y_test))

              precision    recall  f1-score   support

           0       0.58      0.78      0.67    162664
           1       0.67      0.44      0.53    162406

    accuracy                           0.61    325070
   macro avg       0.63      0.61      0.60    325070
weighted avg       0.63      0.61      0.60    325070

              precision    recall  f1-score   support

           0       0.58      0.78      0.67     79926
           1       0.67      0.44      0.53     80184

    accuracy                           0.61    160110
   macro avg       0.63      0.61      0.60    160110
weighted avg       0.63      0.61      0.60    160110

0.4991943039160577
